**Importing Libraries**

In [ ]:
import numpy as np
import math
import random
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

**Dataset Loading and Preprocessing**

In [ ]:
df = pd.read_csv("WA_Fn_UseC_Telco_Customer_Churn.csv",
                 na_values=["No internet service","No phone service"])

df.dropna(inplace=True) #Dropping Rows with NaN
df.dropna(inplace=True, axis='columns') #Dropping Columns with NaN
df.reset_index(drop=True)


df = df.replace({
    "Churn": {
        "Yes" : 1,
        "No" : 0
    }
})

#Dropping the Unnecessary Columns
df.drop(labels=["customerID", "Partner", \
                "StreamingTV", "PhoneService"], 
        axis=1,
        inplace=True)

#Identifying the Categorical and Numerical Columns
categorical_data = pd.DataFrame(df, columns = ["gender", "Dependents", 
                                                  "MultipleLines", 
                                                  "InternetService",
                                                  "OnlineSecurity", 
                                                  "OnlineBackup", 
                                                  "DeviceProtection", 
                                                  "TechSupport", 
                                                  "StreamingMovies", 
                                                  "Contract", 
                                                  "PaperlessBilling", 
                                                  "PaymentMethod", "Churn"])
numerical_data = pd.DataFrame( df, columns = ["SeniorCitizen", "tenure",
                                                "TotalCharges", 
                                                "MonthlyCharges"])   

#defining feature matrix and response vector
data_x = categorical_data.loc[:, categorical_data.columns != 'Churn'] 
data_y = df["Churn"]

#Dropping the Numerical Columns
numerical_data.drop(labels=numerical_data, 
        axis=1,
        inplace=True)

#Converting dataframe into numpy array
categorical_data = np.asarray(categorical_data)


#Split the dataset (80% training, 20% testing) with stratification (using random_state = 911)

train, test = train_test_split(categorical_data, 
                                train_size = 0.8, 
                                stratify = data_y,
                                random_state=911)

**Train and Test set**

In [ ]:
print("Train set: {}".format(train.shape))
print(train[: 5])

print("-------------------------------------------")
print("Test set: {}".format(test.shape))
print(test[: 5])

Train set: (3868, 13)
[['Male' 'No' 'No' 'Fiber optic' 'No' 'No' 'No' 'No' 'Yes'
  'Month-to-month' 'Yes' 'Electronic check' 1]
 ['Male' 'No' 'Yes' 'Fiber optic' 'No' 'No' 'Yes' 'No' 'Yes'
  'Month-to-month' 'Yes' 'Electronic check' 0]
 ['Female' 'No' 'No' 'DSL' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Two year' 'No'
  'Credit card (automatic)' 0]
 ['Female' 'No' 'Yes' 'DSL' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Two year' 'No'
  'Bank transfer (automatic)' 0]
 ['Male' 'Yes' 'Yes' 'DSL' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'Two year' 'No'
  'Mailed check' 0]]
-------------------------------------------
Test set: (967, 13)
[['Female' 'No' 'Yes' 'Fiber optic' 'No' 'Yes' 'Yes' 'No' 'Yes'
  'One year' 'Yes' 'Bank transfer (automatic)' 0]
 ['Female' 'No' 'No' 'Fiber optic' 'No' 'No' 'No' 'No' 'No'
  'Month-to-month' 'Yes' 'Electronic check' 1]
 ['Female' 'No' 'No' 'Fiber optic' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes'
  'Month-to-month' 'Yes' 'Bank transfer (automatic)' 1]
 ['Female' 'No' 'No' 'Fiber optic' 'No' 'No' 'Yes' 'No' 

**Working On Skeleton Code**

In [ ]:
class Node:
    def __init__(self, attribute=None, attribute_values=None, 
                 child_nodes=None, decision=None):
        self.attribute = attribute
        self.attribute_values = attribute_values
        self.child_nodes = child_nodes
        self.decision = decision


class DecisionTree:

    root = None

    @staticmethod
    def plurality_values(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        return Counter(labels).most_common(1)[0][0] #returns -> list of tuple

    @staticmethod
    def all_zero(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        return np.all(labels == 0)

    @staticmethod
    def all_one(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        return np.all(labels == 1)

    @staticmethod
    def importance(data, attributes):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        unique_element, frequency = np.unique(labels, return_counts=True)
        pos = 0
        neg = 0

        if len(unique_element) == 1:
            if unique_element[0] == '0':
                pos = 0
                neg = int(frequency[0])
            elif unique_element[0] == '1':
                pos = int(frequency[0])
                neg = 0
        else:
            neg = int(frequency[0])
            pos = int(frequency[1]) 

        gain = 0 # entropy of parent node
        entropy_child = 0
        if neg == 0:
            gain = -pos*math.log2(pos)
        elif pos == 0:
            gain = -neg*math.log2(neg)
        else:
            gain = -neg*math.log2(neg) - pos*math.log2(pos)

        max_attr = -1
        max_gain = -math.inf
        
        for attr in attributes:
            unique_values = np.unique(data[:, attr])
            for val in unique_values:        
                no_k = pos_k = 0
                for i in range(data.shape[0]):
                    if labels[i] == 0:
                        no_k += 1
                    elif labels[i] == 1:
                        pos_k += 1
                
                if pos_k == 0:
                    entropy_child = -(no_k/(pos_k+no_k)) * math.log2(no_k/(pos_k+no_k))   
                elif no_k == 0:
                    entropy_child = -(pos_k/(pos_k+no_k)) * math.log2(pos_k/(pos_k+no_k))
                else:
                    child_entropy = (-(pos_k/(pos_k+no_k)) * 
                                     math.log2(pos_k/(pos_k+no_k)) - (no_k/(pos_k+no_k)) 
                                     * math.log2(no_k/(pos_k+no_k)))

                gain -= ((pos_k + no_k) / (neg + pos)) * entropy_child        
          
            if  gain > max_gain:
                max_gain = gain
                max_attr = attr    
        
        return max_attr


    def train(self, data, attributes, parent_data):
        data = np.array(data)
        parent_data = np.array(parent_data)
        attributes = list(attributes)
        
        if data.shape[0] == 0:  # if x is empty
            return Node(decision=self.plurality_values(parent_data))

        elif self.all_zero(data):
            return Node(decision=0)

        elif self.all_one(data):
            return Node(decision=1)

        elif len(attributes) == 0:
            return Node(decision=self.plurality_values(data))

        else:
            a = self.importance(data, attributes)
            tree = Node(attribute=a, attribute_values=np.unique(data[:, a]), child_nodes=[])
            attributes.remove(a)
            for vk in np.unique(data[:, a]):
                new_data = data[data[:, a] == vk, :]
                subtree = self.train(new_data, attributes, data)
                tree.child_nodes.append(subtree)

            return tree

    def fit(self, data):
        self.root = self.train(data, list(range(data.shape[1] - 1)), np.array([]))
        
    def predict(self, data):
        predictions = []
        for i in range(data.shape[0]):
            current_node = self.root
            while True:
                if current_node.decision is None:
                    current_attribute = current_node.attribute
                    current_attribute_value = data[i, current_attribute]
                    if current_attribute_value not in current_node.attribute_values:
                        predictions.append(random.randint(0, 1))
                        break
                    idx = list(current_node.attribute_values).index(current_attribute_value)

                    current_node = current_node.child_nodes[idx]
                else:
                    predictions.append(current_node.decision)
                    break

        return predictions



In [ ]:
dc_tree = DecisionTree()
dc_tree.fit(train)
prediction = dc_tree.predict(test)

**Generating Confusion Matrix**

In [ ]:
def confusion_matrix(decesion_list, y_test):
    y_test = np.array(y_test)
    
    tp = tn = fp = fn = 0
    for i in range(len(decesion_list)):
        if decesion_list[i] == 1 and y_test[i] == 1:
            tp += 1
        elif decesion_list[i] == 0 and y_test[i] == 0:
            tn += 1
        elif decesion_list[i] == 0 and y_test[i] == 1:
            fn += 1 
        elif decesion_list[i] == 1 and y_test[i] == 0:
            fp += 1
    return tp, tn, fn, fp

tp, tn, fn, fp = confusion_matrix(prediction, test[:, -1]) 

print("TP = {}, TN = {}, FP = {}, FN = {}".format(tp, tn, fp, fn))

TP = 175, TN = 451, FP = 199, FN = 142


In [ ]:
def precision_score(tp, tn, fn, fp):
    return ( tp / (fp + tp) )
def accuracy_score(tp, tn, fn, fp):
    return  (tp + tn)/ (tp + fn + tn + fp)
def recall_score(tp, tn, fn, fp):
    return tp / (fn + tp)
def f1_score(tp, tn, fn, fp):
    return (2* precision_score(tp, tn, fn, fp) * recall_score(tp, tn, fn, fp) 
            / (precision_score(tp, tn, fn, fp) + recall_score(tp, tn, fn, fp)))


**Performance Measurement Report**

In [ ]:
prec = precision_score(tp, tn, fn, fp)
acc = accuracy_score(tp, tn, fn, fp)
recall = recall_score(tp, tn, fn, fp)
f1 = f1_score(tp, tn, fn, fp)

print("Precision Score = {:10.6f}".format(prec))    
print("Accuracy Score = {:11.6f}".format(acc))    
print("Recall Score = {:13.6f}".format(recall))    
print("F-1 Score = {:16.6f}".format(f1))  

Precision Score =   0.467914
Accuracy Score =    0.647363
Recall Score =      0.552050
F-1 Score =         0.506512
